In [1]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
#df=pd.read_csv("filtered1.csv", dtype='unicode')
df = pd.read_csv('filtered1.csv',
                 dtype={
                       'votes':np.int16,
                       'avg_vote':np.float32,
                       'year':np.int16,
                 })

In [3]:
df

,Unnamed: 0,Unnamed: 0.1,imdb_title_id,title,original_title,year,date_published,genre,duration,country,...,description,avg_vote,votes,budget,usa_gross_income,worlwide_gross_income,metascore,reviews_from_users,reviews_from_critics,combined_features
0,0,4334,tt0035423,Kate & Leopold,Kate & Leopold,2001,2002-03-01,"Comedy, Fantasy, Romance",118,USA,...,An English Duke from 1876 is inadvertedly drag...,6.4,12316,$ 48000000,$ 47121859,$ 76019048,44.0,341.0,115.0,"Comedy, Fantasy, Romance USA James Mangold Meg..."
1,1,14127,tt0064730,Nihon boryoku-dan: Kumicho,Nihon boryoku-dan: Kumicho,2000,1969,"Action, Crime",97,Japan,...,Coming out of jail and hoping for a quiet life...,7.0,168,NaN,NaN,NaN,NaN,3.0,5.0,"Action, Crime Japan Kinji Fukasaku Kôji Tsurut..."
2,2,15675,tt0069049,L'altra faccia del vento,The Other Side of the Wind,2018,2018-11-02,Drama,122,"France, Iran, USA",...,A Hollywood director emerges from semi-exile w...,6.8,5887,NaN,NaN,NaN,78.0,81.0,98.0,"Drama France, Iran, USA Orson Welles John Hust..."
3,3,21806,tt0088751,The Naked Monster,The Naked Monster,2005,2005-04-22,"Comedy, Horror, Sci-Fi",100,USA,...,"A brain-dead sheriff, a stolid secret agent an...",5.4,264,PYG 350000000,NaN,NaN,NaN,15.0,8.0,"Comedy, Horror, Sci-Fi USA Wayne Berwick, Ted ..."
4,4,24217,tt0096056,Delitto e castigo,Crime and Punishment,2002,2002-06-01,Drama,126,"Poland, Russia, USA",...,A modern day adaptation of Dostoyevsky's class...,5.8,633,NaN,NaN,NaN,NaN,13.0,NaN,"Drama Poland, Russia, USA Menahem Golan Crispi..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32625,47131,85849,tt9906644,Manoharam,Manoharam,2019,2019-09-27,"Comedy, Drama",122,India,...,Manoharan is a poster artist struggling to fin...,6.8,491,NaN,NaN,NaN,NaN,9.0,1.0,"Comedy, Drama India Anvar Sadik Vineeth Sreeni..."
32626,47132,85850,tt9908390,Le lion,Le lion,2020,2020-01-29,Comedy,95,"France, Belgium",...,A psychiatric hospital patient pretends to be ...,5.3,398,NaN,NaN,$ 3507171,NaN,NaN,4.0,"Comedy France, Belgium Ludovic Colbeau-Justin ..."
32627,47133,85851,tt9911196,De Beentjes van Sint-Hildegard,De Beentjes van Sint-Hildegard,2020,2020-02-13,"Comedy, Drama",103,Netherlands,...,A middle-aged veterinary surgeon believes his ...,7.7,724,NaN,NaN,$ 7299062,NaN,6.0,4.0,"Comedy, Drama Netherlands Johan Nijenhuis Herm..."
32628,47134,85852,tt9911774,Padmavyuhathile Abhimanyu,Padmavyuhathile Abhimanyu,2019,2019-03-08,Drama,130,India,...,NaN,7.9,265,NaN,NaN,NaN,NaN,NaN,NaN,"Drama India Vineesh Aaradya Anoop Chandran, In..."


In [4]:
#For content-based recommendation we require some features. Here we take features like genre,country, director and actors
#for the recommendation system and store it in a list
df=df.drop(['Unnamed: 0','Unnamed: 0.1','imdb_title_id','date_published','duration','budget','usa_gross_income','worlwide_gross_income','metascore','reviews_from_users','reviews_from_critics','year','avg_vote','votes'],axis=1)
features=['genre','country','director','actors']

In [5]:
#The features selected for the recommendation system are added in the dataframe's coloumn

def combine_features(row): #function for combining the values of these columns into a single string
    return row['genre']+" "+row['country']+" "+row['director']+" "+row['actors']
df=df.replace(np.nan, ' ')#for NaN values
df['combined_features']=df.apply(combine_features,axis=1)
#print("Combined features:"+df['combined_features'].head())
df

,title,original_title,genre,country,language,director,writer,production_company,actors,description,combined_features
0,Kate & Leopold,Kate & Leopold,"Comedy, Fantasy, Romance",USA,"English, French",James Mangold,"Steven Rogers, James Mangold",Konrad Pictures,"Meg Ryan, Hugh Jackman, Liev Schreiber, Brecki...",An English Duke from 1876 is inadvertedly drag...,"Comedy, Fantasy, Romance USA James Mangold Meg..."
1,Nihon boryoku-dan: Kumicho,Nihon boryoku-dan: Kumicho,"Action, Crime",Japan,Japanese,Kinji Fukasaku,"Kinji Fukasaku, Fumio Kônami",Toei Company,"Kôji Tsuruta, Tomisaburô Wakayama, Bunta Sugaw...",Coming out of jail and hoping for a quiet life...,"Action, Crime Japan Kinji Fukasaku Kôji Tsurut..."
2,L'altra faccia del vento,The Other Side of the Wind,Drama,"France, Iran, USA","English, German",Orson Welles,"Orson Welles, Oja Kodar",Royal Road Entertainment,"John Huston, Oja Kodar, Peter Bogdanovich, Sus...",A Hollywood director emerges from semi-exile w...,"Drama France, Iran, USA Orson Welles John Hust..."
3,The Naked Monster,The Naked Monster,"Comedy, Horror, Sci-Fi",USA,English,"Wayne Berwick, Ted Newsom",Ted Newsom,Heidelberg Films,"Kenneth Tobey, Brinke Stevens, R.G. Wilson, Jo...","A brain-dead sheriff, a stolid secret agent an...","Comedy, Horror, Sci-Fi USA Wayne Berwick, Ted ..."
4,Delitto e castigo,Crime and Punishment,Drama,"Poland, Russia, USA","English, Polish",Menahem Golan,"Fyodor Dostoevsky, Menahem Golan",Crime and Punishment Productions Limited,"Crispin Glover, Vanessa Redgrave, John Hurt, M...",A modern day adaptation of Dostoyevsky's class...,"Drama Poland, Russia, USA Menahem Golan Crispi..."
...,...,...,...,...,...,...,...,...,...,...,...
32625,Manoharam,Manoharam,"Comedy, Drama",India,Malayalam,Anvar Sadik,,chakkalakal Films,"Vineeth Sreenivasan, Aparna Das, Basil Joseph,...",Manoharan is a poster artist struggling to fin...,"Comedy, Drama India Anvar Sadik Vineeth Sreeni..."
32626,Le lion,Le lion,Comedy,"France, Belgium",French,Ludovic Colbeau-Justin,"Alexandre Coquelle, Matthieu Le Naour",Monkey Pack Films,"Dany Boon, Philippe Katerine, Anne Serra, Samu...",A psychiatric hospital patient pretends to be ...,"Comedy France, Belgium Ludovic Colbeau-Justin ..."
32627,De Beentjes van Sint-Hildegard,De Beentjes van Sint-Hildegard,"Comedy, Drama",Netherlands,"German, Dutch",Johan Nijenhuis,"Radek Bajgar, Herman Finkers",Johan Nijenhuis & Co,"Herman Finkers, Johanna ter Steege, Leonie ter...",A middle-aged veterinary surgeon believes his ...,"Comedy, Drama Netherlands Johan Nijenhuis Herm..."
32628,Padmavyuhathile Abhimanyu,Padmavyuhathile Abhimanyu,Drama,India,Malayalam,Vineesh Aaradya,"Vineesh Aaradya, Vineesh Aaradya",RMCC Productions,"Anoop Chandran, Indrans, Sona Nair, Simon Brit...",,"Drama India Vineesh Aaradya Anoop Chandran, In..."


In [6]:
#the similarity matrix 
hv=HashingVectorizer(stop_words='english')
matrix=hv.fit_transform(df['combined_features'])

In [7]:
cosineSimilarity=cosine_similarity(matrix)

In [8]:
cosineSimilarity

array([[1.        , 0.        , 0.0270369 , ..., 0.02541643, 0.        ,
        0.        ],
       [0.        , 1.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.0270369 , 0.        , 1.        , ..., 0.02473853, 0.04682929,
        0.02473853],
       ...,
       [0.02541643, 0.        , 0.02473853, ..., 1.        , 0.04402255,
        0.02325581],
       [0.        , 0.        , 0.04682929, ..., 0.04402255, 1.        ,
        0.04402255],
       [0.        , 0.        , 0.02473853, ..., 0.02325581, 0.04402255,
        1.        ]])

In [9]:
indices=pd.Series(df.index, index=df['original_title']).drop_duplicates()
indices[3]

3

In [10]:
def recommend(title,sim_sco=cosineSimilarity):
    x=indices[title]
    sim_sco=list(enumerate(cosineSimilarity[x]))
    sim_sco.sort(key=lambda i:i[1],reverse=True)
    sim_sco=sim_sco[1:11]
    recommended_movie_indices=[i[0] for i in sim_sco]
    return df['original_title'].iloc[recommended_movie_indices]

In [12]:
final=pd.DataFrame(recommend('Justice League'))

In [13]:
final

,original_title
21281,Batman v Superman: Dawn of Justice
8128,Man of Steel
6611,Wonder Woman
7339,Smokin' Aces
29782,Next Gen
7701,Stardust
11321,Seventh Son
14238,Aquaman
6904,In the Name of the King: A Dungeon Siege Tale
9766,Sucker Punch


In [15]:
final.to_csv('Results.csv')